# 3. Microsoft Agent Framework - 멀티 에이전트 워크플로우

이 노트북에서는 **Microsoft Agent Framework**를 사용하여 AI 에이전트를 생성하고 실행하는 방법을 보여줍니다.

## 주요 내용
- 🤖 **ChatAgent**: 단일 에이전트 생성 및 실행
- 🔄 **SequentialBuilder**: 순차적 멀티 에이전트 워크플로우
- 🔧 **@tool**: 도구 정의 및 사용

> 📖 참고: [Agent Framework 문서](https://learn.microsoft.com/en-us/agent-framework/)

---

## 3.1 환경 설정 및 라이브러리 임포트

In [1]:
# 필요한 라이브러리 설치 (최초 1회)
%pip install --quiet agent-framework --pre python-dotenv


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import asyncio
from typing import Annotated
from dotenv import load_dotenv
from pydantic import Field

# Microsoft Agent Framework
from agent_framework import ChatAgent, tool
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import SequentialBuilder


# Azure Identity
from azure.identity import DefaultAzureCredential

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 로드 완료!")

✅ 라이브러리 로드 완료!


In [9]:
# Azure OpenAI Chat Client 설정
AZURE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "")
AZURE_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o")

if AZURE_ENDPOINT:
    chat_client = AzureOpenAIChatClient(
        endpoint=AZURE_ENDPOINT,
        deployment_name=AZURE_DEPLOYMENT,
        credential=DefaultAzureCredential()
    )
    print(f"✅ Azure OpenAI Chat Client 설정 완료!")
    print(f"   엔드포인트: {AZURE_ENDPOINT}")
else:
    chat_client = None
    print("⚠️ AZURE_OPENAI_ENDPOINT를 설정해주세요.")

✅ Azure OpenAI Chat Client 설정 완료!
   엔드포인트: https://ms-foundry-1023.cognitiveservices.azure.com/


## 3.2 도구(Tool) 정의

`@tool` 데코레이터로 에이전트가 사용할 도구를 정의합니다.

In [13]:
@tool(approval_mode="never_require")
def get_weather(
    location: Annotated[str, Field(description="날씨를 조회할 위치")]
) -> str:
    """특정 위치의 날씨 정보를 가져옵니다."""
    print(f"   🌤️ [get_weather] 실행 중...")
    result = f"{location}의 날씨: 맑음, 기온 22°C, 습도 45%"
    print(f"   📤 결과: {result}")
    return result


@tool(approval_mode="never_require")
def search_info(
    query: Annotated[str, Field(description="검색할 쿼리")]
) -> str:
    """정보를 검색합니다."""
    print(f"   🔍 [search_info] 실행 중...")
    result = f"'{query}'에 대한 검색 결과: 관련 정보 3건 발견"
    print(f"   📤 결과: {result}")
    return result


print("✅ 도구 정의 완료!")
print("   - get_weather: 날씨 조회 (실행 로그 포함)")
print("   - search_info: 정보 검색 (실행 로그 포함)")

✅ 도구 정의 완료!
   - get_weather: 날씨 조회 (실행 로그 포함)
   - search_info: 정보 검색 (실행 로그 포함)


## 3.3 단일 에이전트 (ChatAgent)

`ChatAgent`를 생성하고 실행합니다.

In [14]:
# 단일 에이전트 생성 (날씨 + 검색 도구 사용)
simple_agent = ChatAgent(
    name="AssistantAgent",
    chat_client=chat_client,
    instructions="당신은 날씨 정보와 검색 기능을 제공하는 도움이 되는 AI입니다. 사용자의 질문에 적절한 도구를 사용하여 답변하세요.",
    tools=[get_weather, search_info]
)

print("✅ AssistantAgent 생성 완료!")
print("   - get_weather: 날씨 조회")
print("   - search_info: 정보 검색")

✅ AssistantAgent 생성 완료!
   - get_weather: 날씨 조회
   - search_info: 정보 검색


### 단일 에이전트 실행

In [15]:
# 단일 에이전트 실행 (tool 호출 + 실행 결과 표시)
user_query = "서울 날씨 알려주고, 서울 맛집도 검색해줘"
print(f"📍 사용자 질문: '{user_query}'")
print("=" * 60)

tool_calls_detected = []
response_text = ""

async for chunk in simple_agent.run_stream(user_query):
    # 텍스트 응답 수집
    if chunk.text:
        response_text += chunk.text
    
    # Tool 호출 감지 (raw_representation에서 추출)
    if (
        chunk.raw_representation
        and chunk.raw_representation.raw_representation
        and hasattr(chunk.raw_representation.raw_representation, "choices")
        and chunk.raw_representation.raw_representation.choices
        and len(chunk.raw_representation.raw_representation.choices) > 0
    ):
        choice = chunk.raw_representation.raw_representation.choices[0]
        if hasattr(choice, "delta") and hasattr(choice.delta, "tool_calls"):
            tool_calls = choice.delta.tool_calls
            if tool_calls:
                for tc in tool_calls:
                    if tc.function and tc.function.name and tc.function.name not in tool_calls_detected:
                        tool_calls_detected.append(tc.function.name)
                        print(f"\n🔧 Tool 호출: {tc.function.name}")

# 최종 결과 표시
print()
print("=" * 60)
print(f"✅ 호출된 도구: {', '.join(tool_calls_detected) if tool_calls_detected else '없음'}")
print("=" * 60)
print(f"🤖 최종 응답:\n{response_text}")

📍 사용자 질문: '서울 날씨 알려주고, 서울 맛집도 검색해줘'

🔧 Tool 호출: get_weather

🔧 Tool 호출: search_info
   🌤️ [get_weather] 실행 중...
   📤 결과: 서울의 날씨: 맑음, 기온 22°C, 습도 45%
   🔍 [search_info] 실행 중...
   📤 결과: '서울 맛집'에 대한 검색 결과: 관련 정보 3건 발견

✅ 호출된 도구: get_weather, search_info
🤖 최종 응답:
서울의 날씨는 맑음, 기온은 22°C이며, 습도는 45%입니다.

서울 맛집에 관한 검색 결과 3건이 발견되었습니다. 더 구체적인 맛집 정보를 알고 싶다면 말씀해주세요!


## 3.4 SequentialBuilder 워크플로우

`SequentialBuilder`를 사용하여 여러 에이전트를 순차적으로 실행합니다.

```
User Input → Researcher → Writer → Final Output
```

In [17]:
# 멀티 에이전트 생성
researcher = ChatAgent(
    name="Researcher",
    chat_client=chat_client,
    instructions="당신은 연구 전문가입니다. 주어진 주제에 대해 핵심 정보를 요약해주세요.",
    tools=[search_info]
)

writer = ChatAgent(
    name="Writer",
    chat_client=chat_client,
    instructions="당신은 작문 전문가입니다. 제공된 정보를 바탕으로 간결한 글을 작성해주세요."
)

print("✅ 멀티 에이전트 생성 완료!")
print("   - Researcher: 정보 연구")
print("   - Writer: 콘텐츠 작성")


✅ 멀티 에이전트 생성 완료!
   - Researcher: 정보 연구
   - Writer: 콘텐츠 작성


### Sequential 워크플로우 생성 및 실행

In [19]:
# SequentialBuilder로 워크플로우 생성
workflow = SequentialBuilder().participants([researcher, writer]).build()

print("✅ Sequential 워크플로우 생성 완료!")
print("   Researcher → Writer")

✅ Sequential 워크플로우 생성 완료!
   Researcher → Writer


In [31]:
# 워크플로우 실행 (run_stream으로 중간 단계 로그 트레이싱)
user_query = "AI 에이전트의 최신 트렌드에 대해 조사하고 짧은 글로 작성해주세요."
print(f"📍 사용자 질문: '{user_query}'")
print("=" * 60)

final_result = None
current_executor = None

async for event in workflow.run_stream(user_query):
    event_type = type(event).__name__
    
    # 워크플로우 시작
    if event_type == "WorkflowStartedEvent":
        print("\n🚀 워크플로우 시작!")
    
    # Executor 호출 (에이전트 실행 시작)
    elif event_type == "ExecutorInvokedEvent":
        executor_id = getattr(event, 'executor_id', 'Unknown')
        current_executor = executor_id
        
        if executor_id not in ['input-conversation', 'end']:
            print(f"\n🤖 [{executor_id}] 실행 시작...")
            
            # 입력 데이터 표시
            data = getattr(event, 'data', None)
            if data:
                input_text = None
                # 문자열인 경우 (첫 번째 에이전트 - 사용자 질문)
                if isinstance(data, str):
                    input_text = data
                # 리스트인 경우 (ChatMessage 리스트)
                elif isinstance(data, list) and len(data) > 0:
                    first_item = data[0]
                    if hasattr(first_item, 'content'):
                        input_text = first_item.content
                    elif hasattr(first_item, 'text'):
                        input_text = first_item.text
                # AgentExecutorResponse인 경우 (이전 에이전트 결과)
                elif hasattr(data, 'agent_response') and hasattr(data.agent_response, 'text'):
                    input_text = data.agent_response.text
                
                if input_text:
                    display_input = input_text[:100] + "..." if len(input_text) > 100 else input_text
                    print(f"   📥 입력: {display_input}")
    
    # Executor 완료
    elif event_type == "ExecutorCompletedEvent":
        executor_id = getattr(event, 'executor_id', current_executor)
        if executor_id not in ['input-conversation', 'end']:
            print(f"   ✅ [{executor_id}] 완료!")
            
            # data는 리스트, 첫 번째 요소에서 agent_response.text 추출
            data = getattr(event, 'data', None)
            if data and isinstance(data, list) and len(data) > 0:
                first_item = data[0]
                if hasattr(first_item, 'agent_response') and hasattr(first_item.agent_response, 'text'):
                    result_text = first_item.agent_response.text
                    if result_text:
                        display_text = result_text[:200] + "..." if len(result_text) > 200 else result_text
                        print(f"   📄 결과: {display_text}")
    
    # 에이전트 실행 업데이트 (Tool 호출 시 @tool 데코레이터가 print)
    elif event_type == "AgentRunUpdateEvent":
        pass
    
    # 워크플로우 최종 출력
    elif event_type == "WorkflowOutputEvent":
        print("\n" + "=" * 60)
        print("🏁 워크플로우 완료!")
        
        data = getattr(event, 'data', None)
        if data and isinstance(data, list) and len(data) > 0:
            last_msg = data[-1]
            if hasattr(last_msg, 'content'):
                final_result = last_msg.content
            elif hasattr(last_msg, 'text'):
                final_result = last_msg.text

print()
print("=" * 60)
print("📝 최종 결과:")
print("-" * 60)
if final_result:
    print(final_result)
else:
    print("(결과를 가져오지 못함)")

📍 사용자 질문: 'AI 에이전트의 최신 트렌드에 대해 조사하고 짧은 글로 작성해주세요.'

🚀 워크플로우 시작!

🤖 [Researcher] 실행 시작...
   📥 입력: AI 에이전트의 최신 트렌드에 대해 조사하고 짧은 글로 작성해주세요.
   ✅ [Researcher] 완료!
   📄 결과: 2023년 AI 에이전트의 최신 트렌드는 생성형 AI와 자연어 처리 기술을 바탕으로 인간과 더욱 정교하게 소통하며, 멀티모달 AI를 통해 텍스트, 음성, 이미지 데이터를 유기적으로 융합하여 처리하는 방향으로 발전하고 있습니다. 사용자 맞춤형 서비스와 자동화된 의사결정 기능이 강조되며, 윤리적 기술 개발 및 데이터 프라이버시 보호도 주요 화두로 떠오르고 있습...

🤖 [Writer] 실행 시작...
   📥 입력: 2023년 AI 에이전트의 최신 트렌드는 생성형 AI와 자연어 처리 기술을 바탕으로 인간과 더욱 정교하게 소통하며, 멀티모달 AI를 통해 텍스트, 음성, 이미지 데이터를 유기적으로...
   ✅ [Writer] 완료!
   📄 결과: 2023년 AI 에이전트는 생성형 AI와 자연어 처리 기술의 고도화, 멀티모달 데이터 처리를 통해 정교한 사용자 맞춤형 서비스를 제공합니다. 윤리적 설계와 데이터 보호가 강조되며, 여러 산업에서 활용이 급격히 확장되고 있습니다.

🏁 워크플로우 완료!

📝 최종 결과:
------------------------------------------------------------
2023년 AI 에이전트는 생성형 AI와 자연어 처리 기술의 고도화, 멀티모달 데이터 처리를 통해 정교한 사용자 맞춤형 서비스를 제공합니다. 윤리적 설계와 데이터 보호가 강조되며, 여러 산업에서 활용이 급격히 확장되고 있습니다.


In [20]:
# 워크플로우 실행 (각 단계 표시)
print("🚀 워크플로우 시작...")
print("=" * 60)
print()

# Step 1: Researcher 실행
print("📍 Step 1: Researcher 에이전트")
print("   🔍 주제에 대해 정보 검색 중...")
research_result = await researcher.run("AI 에이전트의 최신 트렌드에 대해 조사해주세요.")
print(f"   ✅ 연구 완료!")
print(f"   📄 결과: {str(research_result)[:100]}...")
print()

# Step 2: Writer 실행 (Researcher 결과를 입력으로)
print("📍 Step 2: Writer 에이전트")
print("   ✍️ 연구 결과를 바탕으로 글 작성 중...")
writer_input = f"다음 연구 결과를 바탕으로 짧은 글을 작성해주세요:\n{research_result}"
final_result = await writer.run(writer_input)
print(f"   ✅ 작성 완료!")
print()

# 최종 결과
print("=" * 60)
print("🏁 워크플로우 완료!")
print("=" * 60)
print()
print("📝 최종 결과:")
print("-" * 60)
print(final_result)

🚀 워크플로우 시작...

📍 Step 1: Researcher 에이전트
   🔍 주제에 대해 정보 검색 중...
   🔍 [search_info] 실행 중...
   📤 결과: 'AI 에이전트 최신 트렌드 2023'에 대한 검색 결과: 관련 정보 3건 발견
   ✅ 연구 완료!
   📄 결과: 최신 트렌드 조사 결과, AI 에이전트 분야에서 아래와 같은 주요 동향이 관찰되고 있습니다:

1. **멀티모달 AI**: 텍스트, 이미지, 오디오 등 다양한 데이터를 처리하는 A...

📍 Step 2: Writer 에이전트
   ✍️ 연구 결과를 바탕으로 글 작성 중...
   ✅ 작성 완료!

🏁 워크플로우 완료!

📝 최종 결과:
------------------------------------------------------------
AI 에이전트 분야에서는 멀티모달 AI, 대화형 AI 강화, 개인화된 솔루션이 주요 트렌드로 떠오르고 있습니다. 다양한 데이터를 활용하는 멀티모달 AI 기술이 활발히 개발되고 있으며, 대화형 AI는 사용자와 더 자연스럽고 지속적인 소통을 가능하게 하는 방향으로 진화 중입니다. 또한, 개별 사용자 니즈에 맞춘 맞춤형 AI 솔루션이 주목받고 있습니다. 이와 함께 기술 향상과 확장성, 새로운 애플리케이션 도입 역시 중요한 흐름으로 자리 잡고 있습니다.


## 📋 요약

### 핵심 API

```python
# 1. 설치
pip install agent-framework --pre

# 2. Import
from agent_framework import ChatAgent, tool
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import SequentialBuilder

# 3. 도구 정의
@tool(approval_mode="never_require")
def my_tool(query: str) -> str:
    return f"결과: {query}"

# 4. 단일 에이전트
agent = ChatAgent(
    name="MyAgent",
    chat_client=chat_client,
    instructions="도움이 되는 AI입니다.",
    tools=[my_tool]
)
result = await agent.run("질문")

# 5. Sequential 워크플로우
workflow = SequentialBuilder().participants([agent1, agent2]).build()
result = await workflow.run("작업 요청")
```

---

> 📖 [Agent Framework 문서](https://learn.microsoft.com/en-us/agent-framework/) | [GitHub](https://github.com/microsoft/agent-framework)